## 1. Introduction

Copenhagen is the capital as well as the most populous city of Denmark. As of januar 1st 2020 the city had a population of 794,128 spanning 179,8 km<sup>2</sup>, giving a population density of 4,417/km<sup>2</sup>. Copenhagen is a sprawling metropolis with a large number of venues for entertainment / enjoyment.

Copenhagen can be divided into a number of boroughs. From an investment point of view, if you would like to open a shop / venue, you are of course interested in the price of real estate, as well as the saturation of venues in the area. An investor might look for a cheap location that is not overly saturated by similar venues. From a residential point of view you are again looking for a cheap location, but a large number of venues is now a positive. 

Based on this, it becomes interesting to examine where you get the most bang for your buck - i.e. where an investor can get the cheapest real estate in an area not already saturated by other venues, and where a potential resident can minimize the cost of real estate while maximizing the number of nearby venues, perhaps of a particular category. 

Considering the above problems, it is possible to create a map with information on venue density and real estate price. From this it will be possible to extrapolate where you get the lowest real estate prices combined with the lowest / highest density of certain venues.

## 2. Data

To solve the above problem, two data sets are needed, firstly information on real estate prices for each borough / district is needed, secondly information on venues is needed.

I have below listed how the needed data has been obtained:

<ul>
    <li><b>Real Estate Prices</b> - Finans Danmark releases quarterly information on the average price per square meter for each borough in copenhagen. This dataset is publicly  available on their webpage, ref.: <a href="https://rkr.statistikbank.dk/statbank5a/SelectVarVal/Define.asp?MainTable=BM011">www.rkr.statisbank.dk</a>. I have limited my selection to "Ejerlejlighed" (apartments) and Q3 2020 data. However, in order to later use this data in combination with a map, latitude and longitude is needed for each of the boroughs, this was obtained from Google and merged with the dataset from Finans Danmark. </li>
    <li><b>Venue data set</b> - the Foursquare API will be used to obtain the most popular / most common venues for a given borough in Copenhagen</li>
  
</ul>


## 3. Methodology

As per above, i created a data set containing Borough, DKK pr. m<sup>2</sup>, Latitude and Longitude information for Copenhagen. 

In [4]:
import pandas as pd 
import numpy as np

df = pd.read_excel(r'C:\Users\Malte\OneDrive\Skrivebord\Data Science kursus\Capstone projekt\Copenhagen.xlsx')
df.head()

,Borough,DKK pr. m2,Latitude,Longitude
0,1000-1499 Kbh.K.,56187,55.6842,12.5795
1,1500-1799 Kbh.V.,48701,55.6714,12.5606
2,1800-1999 Frederiksberg C,52644,55.6757,12.5450
3,2000 Frederiksberg,45827,55.6771,12.5133
4,2100 København Ø,47957,55.6761,12.5683


## 3.1 Using the Folium library to visualize the boroughs of Copenhagen.

Based on above dataframe containing the latitude and longitude for the boroughs of Copenhagen, it is possible to visualize these using the Folium Library in Python. The Geopy library will be used to get the longitude and latitude of Copenhagen.

In [5]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.7.0 --yes 
import folium # map rendering library


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [6]:
address = 'Copenhagen'

geolocator = Nominatim(user_agent="Capstone_Project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Copenhagen is {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Copenhagen is 55.6867243, 12.5700724.


With knowledge of the Geographical coordinates of Copenhagen, it is possible, using Folium, to create a map centered on Copenhagen, with the boroughs superimposed on top

In [7]:
#Creating a map focused on copenhagen based on the latitude and longitude obtained above.
map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=12)

# adding a marker for every borough to the map of copenhagen based on the latitude and longitude for each borough.
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_copenhagen)  
    
map_copenhagen

To perform further analysis on the boroughs we can use the Foursquare API as required in the Capstone project. In order to use the Foursquare API we will have to import further libraries as per below. Further, we need to define a function that gets the nearby venues for all the boroughs of Copenhagen.

In [22]:
import json # a library imported in order to handle JSON files

from pandas.io.json import json_normalize # Alows the possibility of tranforming JSON files into a pandas dataframe

# importing the Matplotlib library as well as associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Importing the cluster module from sklearn for use of KMeans clustering.
from sklearn.cluster import KMeans

import requests


print('Libraries imported.')

Libraries imported.


In [23]:
CLIENT_ID = '###' # my Foursquare ID
CLIENT_SECRET = '###' # my Foursquare Secret
VERSION = '20200930' # Foursquare API version - as Q3 data is used for the real estate prices, the Foursquare API version has been set as the last day of Q3 2020

copenhagen_data = df

borough_latitude = copenhagen_data.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = copenhagen_data.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = copenhagen_data.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the Foursquare API URL for the JSON dataset containing venues
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Latitude and longitude values of 1000-1499 Kbh.K. are 55.6842, 12.5795.


With the above function we can now run it to create a new dataframe containing the venues in copenhagen.

In [24]:
copenhagen_venues = getNearbyVenues(names=copenhagen_data['Borough'],
                                   latitudes=copenhagen_data['Latitude'],
                                   longitudes=copenhagen_data['Longitude']
                                  )
print(copenhagen_venues.shape)
copenhagen_venues.head()

1000-1499 Kbh.K.
1500-1799 Kbh.V.
1800-1999 Frederiksberg C
2000 Frederiksberg
2100 København Ø
2150 Nordhavn
2200 København N
2300 København S
2400 København NV
2450 København SV
2500 Valby
2700 Brønshøj
2720 Vanløse
(834, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1000-1499 Kbh.K.,55.6842,12.5795,Kongens Have,55.684361,12.580099,Park
1,1000-1499 Kbh.K.,55.6842,12.5795,Rosenborg Slot,55.685683,12.577482,Palace
2,1000-1499 Kbh.K.,55.6842,12.5795,Davids Samling,55.684271,12.582561,Art Museum
3,1000-1499 Kbh.K.,55.6842,12.5795,Vinværten,55.683427,12.577913,Tapas Restaurant
4,1000-1499 Kbh.K.,55.6842,12.5795,Cinemateket,55.683182,12.578596,Indie Movie Theater


Based on the above dataframe, it is now possible to perform a simple exploratory analysis to identify how many venues there are in each borough.

In [26]:
venues_per_borough = copenhagen_venues.groupby('Borough').count().reset_index()
venues_per_borough['Count'] = venues_per_borough['Venue']
venues_per_borough = venues_per_borough.drop(['Borough Latitude', 'Borough Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category'], axis=1)
venues_per_borough = venues_per_borough.sort_values('Count').reset_index(drop=True)
venues_per_borough.head(13)

,Borough,Count
0,2150 Nordhavn,20
1,2700 Brønshøj,24
2,2450 København SV,31
3,2400 København NV,39
4,2720 Vanløse,51
5,2500 Valby,55
6,2300 København S,56
7,2000 Frederiksberg,58
8,1000-1499 Kbh.K.,100
9,1500-1799 Kbh.V.,100


It is clear that Kbh.K., Kbh.V., Frederiksberg C and København Ø have all reached our limit of 100 venues. A simple analysis could therefore conclude that these are the most 'popular' neighbourhoods. To further examine this, it could be interesting to perform an analysis of the most common venues in each borough. This can be an important metric for both an investor and a potential future resident, as an investor would like to know the saturation of the venue they plan to open, while a potential future resident could have personal preferences towards certain categories of venues.  

In [27]:

# Using one hot encoding to determine which category a given venue is
copenhagen_onehot = pd.get_dummies(copenhagen_venues[['Venue Category']], prefix="", prefix_sep="")

# adding the borough column back to the dataframe
copenhagen_onehot['Borough'] = copenhagen_venues['Borough'] 

# moving the borough column to be the first column
list_column = copenhagen_onehot.columns.tolist()
number_column = int(list_column.index('Borough'))
list_column = [list_column[number_column]] + list_column[:number_column] + list_column[number_column+1:] 
copenhagen_onehot = copenhagen_onehot[list_column]

#grouping the venues by Borough
copenhagen_grouped = copenhagen_onehot.groupby('Borough').mean().reset_index()

#Next up is putting it into a dataframe. 
#In order to get a dataframe showing the most common venues, we first need to write a function that will sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Using our defined function we can now create a dataframe that displays the top 10 most common categories of venues for each borough.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = copenhagen_grouped['Borough']

for ind in np.arange(copenhagen_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(copenhagen_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1000-1499 Kbh.K.,Scandinavian Restaurant,Restaurant,Coffee Shop,Furniture / Home Store,French Restaurant
1,1500-1799 Kbh.V.,Café,Cocktail Bar,Wine Bar,Scandinavian Restaurant,Beer Bar
2,1800-1999 Frederiksberg C,Cocktail Bar,French Restaurant,Scandinavian Restaurant,Italian Restaurant,Pizza Place
3,2000 Frederiksberg,Zoo Exhibit,Pizza Place,Park,Coffee Shop,Supermarket
4,2100 København Ø,Café,Beer Bar,Cocktail Bar,Scandinavian Restaurant,Coffee Shop


In order to later create a map showing information on  DKK pr. m <sup>2</sup> as well as most popular venues we need to create a dataframe containing a column with a combined string with information on the most common venue categories.

In [28]:
top5 = copenhagen_venues.groupby(['Borough','Venue Category']).size().reset_index(name='Counts')
top5 = top5.sort_values(['Borough','Counts'],ascending=False).groupby('Borough').head(5).reset_index(drop=True)

top5['Categories string'] = top5['Counts'].map(str) + " " + top5['Venue Category']
top5 = top5.groupby(['Borough'])['Categories string'].apply(", ".join).reset_index()

top5.head()


,Borough,Categories string
0,1000-1499 Kbh.K.,"7 Scandinavian Restaurant, 5 Coffee Shop, 5 Re..."
1,1500-1799 Kbh.V.,"8 Café, 6 Cocktail Bar, 6 Wine Bar, 3 Beer Bar..."
2,1800-1999 Frederiksberg C,"6 Cocktail Bar, 5 French Restaurant, 5 Italian..."
3,2000 Frederiksberg,"7 Pizza Place, 7 Zoo Exhibit, 4 Park, 3 Coffee..."
4,2100 København Ø,"8 Café, 6 Beer Bar, 6 Cocktail Bar, 4 Coffee S..."


## 4. Results

With knowledge of the most popular venues in each borough, we can now combine these with our table containing the house prices for each borough as well as our table containing the categories as a joined string into a combined table. 

In [34]:
#Create a dataframe with combined data for each borough
combined_table = pd.merge(df,  
                      boroughs_venues_sorted,  
                      on ="Borough",  
                      how ='inner') 

combined_table= pd.merge(combined_table,  
                      top5,  
                      on ="Borough",  
                      how ='inner') 
combined_table= pd.merge(combined_table,  
                      venues_per_borough,  
                      on ="Borough",  
                      how ='inner') 


combined_table.head(13)

,Borough,DKK pr. m2,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Categories string,Count
0,1000-1499 Kbh.K.,56187,55.6842,12.5795,Scandinavian Restaurant,Restaurant,Coffee Shop,Furniture / Home Store,French Restaurant,"7 Scandinavian Restaurant, 5 Coffee Shop, 5 Re...",100
1,1500-1799 Kbh.V.,48701,55.6714,12.5606,Café,Cocktail Bar,Wine Bar,Scandinavian Restaurant,Beer Bar,"8 Café, 6 Cocktail Bar, 6 Wine Bar, 3 Beer Bar...",100
2,1800-1999 Frederiksberg C,52644,55.6757,12.5450,Cocktail Bar,French Restaurant,Scandinavian Restaurant,Italian Restaurant,Pizza Place,"6 Cocktail Bar, 5 French Restaurant, 5 Italian...",100
3,2000 Frederiksberg,45827,55.6771,12.5133,Zoo Exhibit,Pizza Place,Park,Coffee Shop,Supermarket,"7 Pizza Place, 7 Zoo Exhibit, 4 Park, 3 Coffee...",58
4,2100 København Ø,47957,55.6761,12.5683,Café,Beer Bar,Cocktail Bar,Scandinavian Restaurant,Coffee Shop,"8 Café, 6 Beer Bar, 6 Cocktail Bar, 4 Coffee S...",100
5,2150 Nordhavn,58620,55.7165,12.6055,Cruise Ship,Harbor / Marina,Boat or Ferry,Hot Dog Joint,Office,"4 Cruise Ship, 3 Harbor / Marina, 2 Boat or Fe...",20
6,2200 København N,45214,55.6929,12.5478,Café,Coffee Shop,Beer Bar,Wine Bar,Pizza Place,"9 Café, 9 Coffee Shop, 7 Beer Bar, 6 Wine Bar,...",100
7,2300 København S,39966,55.6509,12.5992,Bakery,Pizza Place,Café,Concert Hall,Bar,"5 Bakery, 5 Pizza Place, 4 Café, 3 Bar, 3 Conc...",56
8,2400 København NV,35689,55.7146,12.5287,Pizza Place,Bakery,Gym / Fitness Center,Grocery Store,Thai Restaurant,"6 Pizza Place, 3 Bakery, 2 Beer Bar, 2 Grocery...",39
9,2450 København SV,43904,55.6491,12.5353,Grocery Store,Hotel,Pizza Place,Café,Restaurant,"2 Café, 2 Grocery Store, 2 Hotel, 2 Pizza Plac...",31


Having combined the table, we can now create a map containing information on the DKK pr. m<sup>2</sup> as well as the most popular venues in the given boroughs.

In [31]:
#Creating a map focused on copenhagen based on the latitude and longitude obtained above.
address = 'Copenhagen'

geolocator = Nominatim(user_agent="Capstone_Project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_copenhagen = folium.Map(location=[latitude, longitude], zoom_start=12)

# adding a marker for every borough to the map of copenhagen based on the latitude and longitude for each borough.Each marker contains the borough name, dkk pr. m2 and most popular venue categories.
for lat, lng, borough, DKK, string in zip(combined_table['Latitude'], combined_table['Longitude'], combined_table['Borough'], combined_table['DKK pr. m2'], combined_table['Categories string']):
    label = folium.Popup(str(borough) + " / " + "Price pr. square meter: " + str(DKK) + " / " + "Most common venues: " + str(string), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_copenhagen)  
    
map_copenhagen

From the map we can, as an example, see that the average price pr. square meter for an apartment in 2200 København N amounts to 45.214, with the most common venues in the borough being:

<ol>
  <li>Café (with 9 venues)</li>
  <li>Coffee Shop (also 9 venues)</li>
  <li>Beer Bar (with 7 venues)</li>
  <li>Wine Bar (with 6 venues)</li>
  <li>Pizza Place (with 5 venues)</li>
</ol>

## 5. Discussion

In [35]:
Kbh.K., Kbh.V., Frederiksberg C, København Ø and København N are the most ‘popular’ boroughs with the most venues, with all of them reaching the limit of 100 venues. This is in line with what one would expect, as these are the most downtown areas of Copenhagen. If all a private resident values is the number of venues, it can then be extrapolated that you get the most bang for the buck in 2200 København N, as this has the lowest price per square meter in the above listed boroughs.

On the other hand, if price is most important, it is clear that 2700 Brønshøj is the cheapest area to live, however the number venues has also decreased drastically, even more so proportionally than the price, why it can be observed that there is not a linear relationship between price and number of venues. 

While no specific recommendation can be made based on the tables and maps for a future resident, it enables a potential resident to make an informed decision about where to live. 

From an investment perspective, It can, for example, be observed that the downtown areas of Copenhagen has significantly more bars than the outskirts of Copenhagen, where grocery store are more common. As a specific example the most common venue in 1800-1999 Frederiksberg are cocktail bars (a total 6 in the borough), where for 2720 Vanløse the most common venue is pizza places (6 venues in the borough). 

It can thus be extrapolated that if an investor wants to differentiate themselves from the competition, they should perhaps aboid opening a cocktail bar, French restaurant, Italian restaurant or the like in 1800-1999 Frederiksberg. However, of course an assumption can also be made, that there is a correlation between the types of venues in a given borough and the demographic of the borough, why it is a balancing act to differentiate your venue sufficiently from others, while also being surrounded by the appropriate demographic. Similar observations can be made for every borough. 

As with potential future residents, no specific recommendation for investors can be made, however the tables and maps enables an investor to make a decision on an informed basis. 


SyntaxError: invalid syntax (<ipython-input-35-9ae509d80180>, line 1)